### Import data

In [12]:
import urllib.request 

url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")

file_path = "the-verdict.txt"

urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x108ffdac0>)

### Read data

In [13]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("The total number of characters in the text is: ", len(raw_text))
print("The first 100 characters in the text are:")
print(raw_text[:99])
    


The total number of characters in the text is:  20479
The first 100 characters in the text are:
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### Preprocess text (tokenization)


In [14]:
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text) # Split text into tokens

preprocessed = [item.strip() for item in preprocessed if item.strip()] # Remove empty strings
print(len(preprocessed)) # Print the number of tokens
print(preprocessed[:30]) # Print the first 10 tokens

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### Convert tokens into token IDs



In [15]:
all_words = set(preprocessed)
vocab_size = len(all_words)
print("The total number of unique tokens in the raw text is: ", vocab_size)

The total number of unique tokens in the raw text is:  1130


In [18]:
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50: # Print only the first 50 tokens
        break 

('question', 0)
('appearance', 1)
('borne', 2)
('anything', 3)
('travelled', 4)
('disguised', 5)
('buying', 6)
('Now', 7)
('upstairs', 8)
('straight', 9)
('up-stream', 10)
('Hang', 11)
('look', 12)
('hot-house', 13)
('half-mechanically', 14)
('just', 15)
('poised', 16)
('about', 17)
('care', 18)
('our', 19)
('object', 20)
('cards', 21)
('failed', 22)
('hardly', 23)
('dining-room', 24)
('self-confident', 25)
('vista', 26)
('chap', 27)
('.', 28)
('Professional', 29)
('slight', 30)
('grew', 31)
('embarrassed', 32)
('quietly', 33)
('shall', 34)
('fancy', 35)
('deploring', 36)
('dead', 37)
('modest', 38)
('domestic', 39)
('insensible', 40)
('marble', 41)
('t', 42)
('straddling', 43)
('and', 44)
('had', 45)
('excuse', 46)
('equanimity', 47)
('doesn', 48)
('garlanded', 49)
('showed', 50)
